In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from tqdm.notebook import tqdm

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = True

# '2Min'

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

In [6]:
%%time

def get_classifiers():
    clss = classifiers = [
        lambda : make_pipeline(StandardScaler(),
                                MLPClassifier(
                                    solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
                                    early_stopping=True)
                                ),
        
        #lambda : xgb.XGBClassifier(random_state=1,
        #                           objective = "binary:logistic",
        #                           eval_metric='mlogloss',
        #                           learning_rate=0.01),
        
        lambda : GaussianNB(),
        #lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        
        #lambda : LogisticRegression(),
        #lambda : KNeighborsClassifier(3),
        #lambda : GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        #lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        
        lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    #clss = []
    clss.append(lambda : rand())
    return clss


estimators = [
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        models.append(cls())
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11 µs


In [7]:
get_all_models_factory = lambda : get_all_models()

In [8]:
%%time

def test_models(provider, get_all_models_factory, steps = [1]):
    
    score_board_by_step = {}

    for step in steps:
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            train_by_step(model, step, provider)
            
            result = run_trial(model, provider, step)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 12 µs, sys: 1 µs, total: 13 µs
Wall time: 15.3 µs


In [9]:

steps = [1, 3]

windows = [5, 10, 20, 30, 40]

minutes = [15, 5, 3, 1]


In [10]:

def test_windows(minutes, steps = steps):
    results = []
    for win in tqdm(windows):
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        online = get_online_data(minutes, sourceDataGenerator, load_from_disk, win)
        result = test_models(
            online, 
            get_all_models_factory,
            steps = steps
        )
        for rank in result:
            rank['window'] = win
            rank['minutes'] = minutes
            
        results.append(result)
    return results
    

In [11]:
%%time

min_results = []

for minu in minutes:
    print(f"Evaluating {minu} min")
    results = test_windows(minu, steps)
    min_results.append(results)


Evaluating 15 min


  0%|          | 0/5 [00:00<?, ?it/s]

Step 1


  0%|          | 0/25 [00:00<?, ?it/s]

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/minicon

...

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/minicon

Step 3


  0%|          | 0/25 [00:00<?, ?it/s]

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samirantonio/opt/minicon

CPU times: user 18h 6min 18s, sys: 1h 24min 12s, total: 19h 30min 30s
Wall time: 12h 43min 44s


In [12]:
model_path = f'model/all_results'
dump(min_results, model_path)

['model/all_results']

In [171]:
best_results = []
for results in min_results:
    for r in results:
        if (len(r) > 0):
            for win_result in r:
                profit = win_result['profit']
                model = win_result['model']
                window = win_result['window']
                minutes = win_result['minutes']
                step = win_result['step']
                profit = win_result['profit']
                models_score = win_result['models_score']
                #print(models_score)
                if (models_score['btcusd']['recall'] > 0.5):
                    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
                    print(f"{win_result['model']}")
                    print(f"")
                    best_results.append(win_result)
        else:
            print("Not found")

print(f"{len(best_results)} selected")

Minutes=15 Window=20 Step=3 | 94.5318451074584
Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(random_state=10),
                     n_features_to_select=8)),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=20000, random_state=1,
                                                solver='lbfgs'))]))])

Minutes=15 Window=30 Step=3 | 96.64697605648108

Minutes=15 Window=30 Step=3 | 92.84149484201407
Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                ('m',
                 <catboost.core.CatBoostClassifier object at 0x7f7ea53c4970>)])

Minutes=15 Window=40 Step=3 | 99.59019423481018
Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(random_state=10),
                

In [131]:
model_path = f'model/results'
dump(best_results, model_path)

['model/results']

In [146]:
def print_result(result):
    model = result['model']
    window = result['window']
    minutes = result['minutes']
    step = result['step']
    profit = result['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit} {best['models_profit_metric']}")
    print(f"{model}")

In [148]:
def myFunc(e):
    #return e['profit']
    return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = False)

for best in best_results:
    print_result(best)
    model_path = f'model/result_{window}_{minutes}_{step}'
    dump(best, model_path)
    print(f"")

Minutes=1 Window=40 Step=3 | 100.0219338939727 {'btcusd': 0.0033336885208469697}
Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(random_state=10),
                     n_features_to_select=8)),
                ('m',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=1 Window=10 Step=3 | 100.03141705428982 {'btcusd': 0.004779809112081957}
Pipeline(steps=[('s', RFECV(estimator=RandomForestClassifier(random_state=10))),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=20000, random_state=1,
                                                solver='lbfgs'))]))])

Minutes=1 Window=20 Step=3 | 100.04279574219336 {'btcusd': 0.006515


Minutes=1 Window=10 Step=3 | 100.3244872722791 {'btcusd': 0.049396434932184555}
Pipeline(steps=[('s', RFECV(estimator=RandomForestClassifier(random_state=10))),
                ('m',
                 <catboost.core.CatBoostClassifier object at 0x7f7d9d388610>)])

Minutes=1 Window=10 Step=3 | 100.33403847439905 {'btcusd': 0.05084255552341954}

Minutes=1 Window=10 Step=1 | 100.23892584218946 {'btcusd': 0.05169988530372872}
GaussianNB()

Minutes=1 Window=10 Step=3 | 100.34236373484065 {'btcusd': 0.05212123057251153}
Pipeline(steps=[('s',
                 RFE(estimator=RandomForestClassifier(random_state=10),
                     n_features_to_select=10)),
                ('m',
                 <catboost.core.CatBoostClassifier object at 0x7f7d770e5f10>)])

Minutes=1 Window=40 Step=3 | 100.41927031431835 {'btcusd': 0.06382719620050843}
Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                ('m',
                 <catboost.core.CatBoostClassifier o

In [149]:
def myFunc(e):
    return e['profit']

best_results.sort(key=myFunc, reverse = True)

In [159]:
best_results[0]

{'profit': 101.23717129810005,
 'model': Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                 ('m',
                  RandomForestClassifier(criterion='entropy', max_depth=100,
                                         n_estimators=30, random_state=42))]),
 'models_profit': {'btcusd': 1.2372},
 'models_score': {'btcusd': {'recall': 0.03571428571428571,
   'precision': 0.07692307692307693,
   'f1': 0.04878048780487805,
   'accuracy': 0.9219219219219219,
   'roc_auc': 0.5051317982123922}},
 'models_profit_metric': {'btcusd': 0.18833056794483433},
 'reference_profit': {'btcusd': 6.5693},
 'step': 3,
 'window': 5,
 'minutes': 1}

In [151]:
def load_online(minutes, window, val_end = 1626388860, currency = []):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 999,
                 val_keys = currency,
                 #val_start = val_start,
                 val_end = val_end,
                 train_start_list = []
    )

    online.load_cache()
    return online

In [152]:
def backtest_model(model, x, closed_prices, back):
    
    for idx in range(len(x)):
        xx = [x[idx]]
        yy = model.predict(xx)[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    
    #print(f"Closing backtest_model {price} {back.current}")
    back.sell(price)
    #print(f"Closed backtest_model {back.current}")
    
    return back

In [153]:
def test_model(model, set_key, provider, step, verbose = True):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    #print(len(x))

    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = verbose, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [154]:
from datetime import datetime


timestamp = int(datetime.timestamp((datetime.now())))


#timestamp = 1610959352

In [189]:
online_cache = {}

In [ ]:
scoreboard = []

fails = []

currency = "ltcusd"
currency = "ethusd"
currency = "batusd"

currs = ["batusd", "ltcusd", "ethusd", "btcusd"]
#currs = ["btcusd"]

for best in best_results:
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    cache_key = f"{minutes}-{window}"
    try:
        online = online_cache[cache_key]
    except :
        print(f"Not found {cache_key}")
        online = load_online(minutes = minutes, window = window, val_end = timestamp, currency = currs)
        online_cache[cache_key] = online
        
    profits = []
    backs = {}
    score = {}
    for train_set in online.val_keys:
        back = test_model(model, train_set, online, step, False)
        profits.append(back.get_profit())
        backs[train_set] = back
        
    score['profit'] = np.average(profits)
    score['backs'] = backs
    score['result'] = best
    scoreboard.append(score)

    


Minutes=15 Window=20 Step=3 | 94.5318451074584
Not found 15-20
https://www.bitstamp.net/api/v2/ohlc/batusd/ -> {'currency_pair': 'batusd', 'step': 900, 'limit': 999, 'start': 1623435037, 'end': 1626787422}
Downloaded from 2021-07-10 04:45:00 to 2021-07-20 14:15:00 0.46580
Resample 15Min - TecAn ( windows 20, windows_limit 100 )
Processing 998 of batusd


  0%|          | 0/998 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/ltcusd/ -> {'currency_pair': 'ltcusd', 'step': 900, 'limit': 999, 'start': 1623435037, 'end': 1626787422}
Downloaded from 2021-07-10 04:45:00 to 2021-07-20 14:15:00 106.73
Resample 15Min - TecAn ( windows 20, windows_limit 100 )
Processing 998 of ltcusd


  0%|          | 0/998 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/ethusd/ -> {'currency_pair': 'ethusd', 'step': 900, 'limit': 999, 'start': 1623435037, 'end': 1626787422}
Downloaded from 2021-07-10 04:45:00 to 2021-07-20 14:15:00 1754.07
Resample 15Min - TecAn ( windows 20, windows_limit 100 )
Processing 998 of ethusd


  0%|          | 0/998 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 900, 'limit': 999, 'start': 1623435037, 'end': 1626787422}
Downloaded from 2021-07-10 04:45:00 to 2021-07-20 14:15:00 29717.66
Resample 15Min - TecAn ( windows 20, windows_limit 100 )
Processing 998 of btcusd


  0%|          | 0/998 [00:00<?, ?it/s]

In [ ]:
profs = []
for score in scoreboard:
    profit = score['profit']
    result = score['result']
    backs = score['backs']
    print(f"{profit}")
    for back in backs:
        print(f"{back} -> {backs[back].get_profit()}")
    print_result(result)
    profs.append(profit)

    
print()
print(f"{np.average(profs)} ")